In [ ]:
import pandas as pd
import numpy as np
from battery_plot import *

In [ ]:
data = "JMFM_17_SOC_OCV_94plus5_Test_220624.txt"

In [ ]:
df = pd.read_csv(data, skiprows=[0,1,2],sep="\t")

In [ ]:
def get_sec(time_str):
    d, m, s = time_str.split(":")
    d, h = d.split("d")
    d = int(d)
    h = int(h)
    m = int(m)
    s = int(s)
    return d*86400 + h*3600 + m*60 +s

def nf_soc_compute(data, capacity, to_csv_pts = False, to_csv_curve = False):
    '''
    .txt file[, bool, bool] -> pd.DataFrame, pd.DataFrame[, .csv file, .csv file] 
    Precondition, data is a .txt file from the Maccor
    
    Parameters:
    `capacity` float
        the capacity of the LFP cell
    `to_csv_pts` bool
        to send the extracted points to a csv file
    `to_csv_curve` bool
        to send the estimated points to a csv file
    '''
    data = pd.read_csv(data, skiprows=[0,1,2], sep="\t")
    
    first_idx = data[data["MD"] == "D"].iloc[0].name
    
    df = data.loc[first_idx-1:,["Test Time", "Current", "Voltage", "MD"]].copy()
    
    df.columns = [col.lower() for col in df.columns]

    df.loc[df["md"] == "D","current"] *= -1

    delta_cap = df["current"] / 3600

    df["soc"] = list((capacity + delta_cap.cumsum()) / capacity * 100)
    df.loc[:,"test time (sec)"] = [get_sec(i) for i in df["test time"]]
    
#     if to_csv_pts:
#         name = input("Name of output file for SOC-OCV points: ")
#         data.to_csv(name)
#         print( f"\n{name} was created.\n" )
#     if to_csv_curve:
#         name = input("Name of output file for SOC-OCV curve: ")
#         curve.to_csv(name)
#         print( f"\n{name} was created.\n" )

    mask = df["test time (sec)"].diff()
    mask.iloc[0],mask.iloc[-1] = 1.0,1.0
    df["delta t"] = mask
    
    return df[["test time (sec)","delta t","current","voltage","soc"]]

In [ ]:
df = nf_soc_compute(data, 18.208)

In [ ]:
df.to_csv("JMFM_17_SOC_OCV_94plus5_Test_220624_soc.csv", index=False)

In [ ]:
data_plot(data = [df],
          x = ["test time (sec)"],
          y = [ "soc" ],
          title = "JMFM-17 OCV vs. SOC Curve",
          x_title = "SOC (%)",
          y_title = "OCV (V)",
          mode = "lines",
          color = "darkorchid",
          name = None,
          size = None
         )